### app 만들기
```
python manage.py startapp guestbook
```

### config/settings.py
```config/settings.py
INSTALLED_APPS = [
    ...
    'guestbook',
]

```

### config/urls.py
```config/urls.py
from django.urls import include
...
urlpatterns = [
    ...
    path('guestbook/', include('guestbook.urls')),

]

```

#### guestbook/models.py
```guestbook/models.py
from django.db import models
from datetime import datetime

class Guestbook(models.Model):
    idx = models.AutoField(primary_key=True)
    name = models.CharField(null=False, max_length=50)
    email = models.CharField(null=False, max_length=50)
    passwd = models.CharField(null=False, max_length=50)
    content = models.TextField(null=False)
    post_date = models.DateTimeField(default=datetime.now, blank=True)

```

#### guestbook/urls.py
```guestbook/urls.py
from django.urls import path
from guestbook import views

urlpatterns = [
    path('',views.list),
    path('write',views.write),
    path('gb_insert',views.insert),
    path('passwd_check',views.passwd_check),
    path('gb_update',views.update),
    path('gb_delete',views.delete),
    ]
```

#### guestbook/views.py
``` guestbook/views.py
from django.shortcuts import render, redirect
from django.db.models import Q
from guestbook.models import Guestbook

# Create your views here.
def list(request):
    try:
        searchkey = request.POST['searchkey']
    except:
        searchkey = 'name'
    try:
        search = request.POST['search']
    except:
        search = ''

    if searchkey == 'name_content':
        gbList = Guestbook.objects.filter(Q(name__contains=search) | Q(content__contains=search)).order_by('-idx')
    elif searchkey == 'name':
        gbList = Guestbook.objects.filter(Q(name__contains=search)).order_by('-idx')
    elif searchkey == 'content':
        gbList = Guestbook.objects.filter(Q(content__contains=search)).order_by('-idx')

    try:
        msg = request.GET['msg']
    except:
        msg = ''
    return render(request, 'guestbook/list.html',
                  {'gbList': gbList, 'gbCount': len(gbList), 'searchkey': searchkey,
                   'search': search, 'msg': msg})

def write(request):
    return render(request, 'guestbook/write.html')

def insert(request):
    row = Guestbook(name=request.POST['name'], email=request.POST['email'],
                    passwd=request.POST['passwd'], content=request.POST['content'])
    row.save()
    return redirect('/guestbook')

def passwd_check(request):
    id = request.POST['idx']
    pwd = request.POST['passwd']
    row = Guestbook.objects.get(idx=id)
    if row.passwd == pwd:
        return render(request, 'guestbook/edit.html')
    else:
        return redirect('/guestbook/?msg=error')

def update(request):
    id = request.POST['idx']
    pwd = request.POST['passwd']
    row = Guestbook.objects.get(idx=id)
    if row.passwd == pwd:
        row = Guestbook(idx=id, name=request.POST['name'], email=request.POST['email'],
                        passwd=pwd, content=request.POST['content'])
        row.save()
        return redirect('/guestbook')
    else:
        return redirect('/guestbook/?msg=error')

def delete(request):
    id = request.POST['idx']
    Guestbook.objects.get(idx=id).delete()
    return redirect('/guestbook')
```

#### mytest/templates/index.html
```mytest/templates/index.html
...
<a href="guestbook/">방명록</a><br>
```


#### guestbook/templates/guestbook/list.html
```guestbook/templates/guestbook/list.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  {% if msg == 'error' %}
  <script>alert('비밀번호를 확인하세요')</script>
  {% endif %}
</head>
<body>
<h2>방명록</h2>
{{gbCount}}개의 글이 있습니다.
<form name="form1" method="post">
{% csrf_token %}
    <select name="searchkey">
        {% if searchkey == 'name' %}
            <option value="name">이름</option>
            <option value="content">내용</option>
            <option value="name_content">이름+내용</option>
        {% elif searchkey == 'content' %}
            <option value="name">이름</option>
            <option value="content" selected>내용</option>
            <option value="name_content">이름+내용</option>
        {% else %}
        <option value="name">이름</option>
        <option value="content">내용</option>
        <option value="name_content" selected>이름+내용</option>
        {% endif %}
    </select>
    <input name="search" value="{{search}}">
    <input type="submit" value="조회">
</form>
<input type="button" value="Home" onclick="location.href='/'">
<input type="button" value="글쓰기" onclick="location.href='/guestbook/write'">
{% for row in gbList %}
<form method="post" action="/guestbook/passwd_check">
    {% csrf_token %}
    <input type="hidden" name="idx" value="{{row.idx}}">
    <table border="1" width="600px">
        <tr>
            <td>이름</td>
            <td>{{row.name}}</td>
            <td>날짜</td>
            <td>{{row.post_date | date:'Y-m-d'}}</td>
        </tr>
        <tr>
            <td>이메일</td>
            <td colspan="3">{{row.email}}</td>
        </tr>
        <tr>
            <td colspan="4">{{row.content}}</td>
        </tr>
        <tr>
            <td colspan="4">
                비밀번호 <input type="password" name="passwd">
                <input type="submit" value="수정/삭제">
            </td>
        </tr>
    </table>
</form>
{% endfor %}
</body>
</html>
```

#### guestbook/templates/guestbook/write.html
```guestbook/templates/guestbook/write.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  <script>
    function check(){
      document.form1.action='gb_insert';
      document.form1.submit();
      }
  </script>
</head>
<body>
<h2>방명록 작성</h2>
<form name="form1" method="post">
  {% csrf_token %}
  <table border="1" width="500px">
    <tr>
      <td>이름</td>
      <td><input name="name" size="40"></td>
    </tr>
    <tr>
      <td>이메일</td>
      <td><input name="email" size="40"></td>
    </tr>
    <tr>
      <td>비밀번호</td>
      <td><input type="password" name="passwd" size="40"></td>
    </tr>
    <tr align="center">
      <td colspan="2"><textarea rows="5" cols="55" name="content"></textarea></td>
    </tr>
    <tr align="center">
      <td colspan="2">
        <input type="button" value="확인" onclick="check()">
      </td>
    </tr>
  </table>
</form>
</body>
</html>
```



#### guestbook/templates/guestbook/edit.html
```guestbook/templates/guestbook/edit.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  <script>
    function gb_update(){
      document.form1.action='gb_update';
      document.form1.submit();
    }
    function gb_delete(){
      if(confirm("삭제하시겠습니까?")){
        document.form1.action='gb_delete';
        document.form1.submit();
      }
    }
  </script>
</head>
<body>
<h2>방명록 편집</h2>
<form name="form1" method="post">
{% csrf_token %}
<table border="1" width="500px">
  <tr>
    <td>이름</td>
    <td><input name="name" size="40" value="{{row.name}}"></td>
  </tr>
  <tr>
    <td>이메일</td>
    <td><input name="email" size="40" value="{{row.email}}"></td>
  </tr>
  <tr>
    <td>비밀번호</td>
    <td><input type="password" name="passwd" size="40"></td>
  </tr>
  <tr align="center">
    <td colspan="2"><textarea rows="5" cols="55" name="content">{{row.content}}</textarea></td>
  </tr>
  <tr align="center">
    <td colspan="2">
      <input type="hidden" name="idx" value="{{row.idx}}">
      <input type="button" value="수정" onclick="gb_update()">
      <input type="button" value="삭제" onclick="gb_delete()">
      <input type="button" value="목록" onclick="location.href='/guestbook'">
    </td>
  </tr>
</table>
</form>
</body>
</html>


```
